# Validation of ES-QC for the year 2021

In [1]:
import pandas as pd
import os
from energyscope.models import Model
from energyscope.energyscope import Energyscope
from energyscope.result import postprocessing

In [2]:
# AMPL licence 
path_to_ampl_licence = r'C:\Users\matth\ampl' # Path to the AMPL licence file
os.environ['PATH'] = path_to_ampl_licence+':'+os.environ['PATH']

In [3]:
path_model = '../02_Model/'
path_data = '../02_Model/2020/'

In [4]:
max_indicator = pd.read_csv('../02_Model/2020/QC_techs_lca_max.csv')

## Initialize the model

In [5]:
with open(path_model + 'QC_objective_function.mod', 'w') as f:
        f.write('minimize obj: TotalCost;')

In [6]:
# Initialize the 2021 QC model with .mod and .dat files
model = Model(
    mod_files=[
        path_model+'QC_es_main.mod',
        path_model+'QC_objective_function.mod',
        path_data+'QC_objectives_lca.mod',
        path_data+'QC_objectives_lca_direct.mod',
    ],
    dat_files=[
        path_data+'QC_data.dat',
        path_data+'QC_mob_techs_dist_B2D.dat',
        path_data+'QC_techs_B2D.dat',
        path_data+'QC_mob_params.dat',
        path_data+'QC_validation.dat',
        path_data+'QC_techs_lca.dat',
        path_data+'QC_techs_lca_direct.dat',
    ],
)

In [7]:
# Define the solver options
solver_options = {
    'solver': 'gurobi',
    'solver_msg': 0,
}

In [8]:
# Initialize the EnergyScope model
es = Energyscope(model=model, solver_options=solver_options)

In [9]:
# Solve the model and get results
results = postprocessing(es.calc())

Gurobi 12.0.0: 

## Validation

In [10]:
# Annual production
annual_prod = results.variables['Annual_Prod']

In [11]:
# Installed capacities 
installed_capacities = results.variables['F_Mult']

In [12]:
# Annual resources
annual_ressources = results.variables['Annual_Res']

In [13]:
# Validation metrics extracted from StatCan and "État de l'énergie au Québec 2024" (data for 2020)
validation_dict = {
    'WIND_ONSHORE': 11122, # onshore wind production [GWh]
    'HYDRO': 201250, # hydro production [GWh]
    'PV': 14, # PV production [GWh]
    'ELECTRICITY_EHV': 32778, # electricity imports [GWh]
    'NG_EHP': 59867, # natural gas imports [GWh]
    'COAL': 3991, # coal and coke imports [GWh]
    'LFO': 28663, # oil imports for agriculture, industry and buildings [GWh]
    'DIESEL': 34046, # diesel imports [GWh]
    'GASOLINE': 69019, # gasoline imports [GWh]
    'TotalGWP': 48980, # total GWP [kt CO2]
    'JETFUEL': 8470, # jet fuel imports [GWh]
    'WOOD': 29587, # wood production [GWh]
}

In [14]:
def compare(
        tech = None, 
        res = None, 
        emissions = None,
        n_digits = 3,
):
    if tech is not None:
        data_type = tech
        if tech == 'HYDRO':
            annual = annual_prod.loc['HYDRO_DAM'].Annual_Prod + annual_prod.loc['HYDRO_RIVER'].Annual_Prod # sum of hydro production from dams and rivers
        else:
            annual = annual_prod.loc[tech].Annual_Prod
    
    elif res is not None:
        data_type = res
        annual = annual_ressources.loc[res].Annual_Res
    
    elif emissions is not None:
        data_type = emissions
        direct_emissions = results.variables['DIRECT_op'].reset_index()
        direct_emissions = direct_emissions[direct_emissions.index0 == 'm_CCS']
        annual = direct_emissions.DIRECT_op.sum() * max_indicator[max_indicator.Abbrev == 'm_CCS'].max_AoP.iloc[0]
    
    else:
        raise ValueError("Provide either a technology or a resource to calculate the percentage difference")
    
    return [
        data_type, # technology or resource
        round(validation_dict[data_type]/1000, n_digits), # reference value in TWh or Mt CO2
        round(float(annual)/1000, n_digits), # model value in TWh or Mt CO2
        round(validation_dict[data_type]/1000 - float(annual)/1000, n_digits), # absolute difference in TWh or Mt CO2
        round((validation_dict[data_type] - float(annual)) / validation_dict[data_type], n_digits), # relative difference
    ]

In [15]:
data = []
for res in ['NG_EHP', 'LFO', 'COAL', 'DIESEL', 'GASOLINE', 'JETFUEL', 'WOOD', 'ELECTRICITY_EHV']:
    data.append(compare(res=res))
# Technologies
for tech in ['HYDRO', 'WIND_ONSHORE', 'PV']:
    data.append(compare(tech=tech))
# Emissions
for emissions in ['TotalGWP']:
    data.append(compare(emissions=emissions))

df = pd.DataFrame(data, columns=['TECH', 'REF', 'ES', 'DELTA', 'RELATIVE_DELTA'])

In [16]:
df.to_csv('../03_Results/Tables/soo_reference/validation_table.csv', index=False)

In [17]:
df

,TECH,REF,ES,DELTA,RELATIVE_DELTA
0,NG_EHP,59.867,59.209,0.658,0.011
1,LFO,28.663,28.570,0.093,0.003
2,COAL,3.991,4.015,-0.024,-0.006
3,DIESEL,34.046,31.763,2.283,0.067
4,GASOLINE,69.019,69.153,-0.134,-0.002
5,JETFUEL,8.470,7.655,0.815,0.096
6,WOOD,29.587,26.835,2.752,0.093
7,ELECTRICITY_EHV,32.778,35.211,-2.433,-0.074
8,HYDRO,201.250,201.492,-0.242,-0.001
9,WIND_ONSHORE,11.122,11.456,-0.334,-0.030
